## main

In [1]:
def lim(x, lolim=0, uplim=21):
    '''将得分限制在lolim～uplim'''
    if x > uplim: return uplim
    if x < lolim: return lolim
    return x

def get_score_1(p1_score, p2_score, p1_res, p2_res, _print=False):
    '''
    # 用于计算KIAA羽毛球俱乐部积分赛的积分（单打）
    # 分数范围：0～200
    # rank = 分数 / 10
    # Parameters
        p1_score: 比赛前P1的分数
        p2_score: 比赛前P2的分数
        p1_res  : 对局中P1的得分
        p2_res  : 对局中P2的得分
    # Returns:
        p1_score_new: 比赛后P1的分数
        p2_score_new: 比赛后P2的分数
    '''
    
    def K(score):
        '''放大系数'''
        return 2 - 0.01 * score
    
    # 超过 20 分计为 20
    p1_res = lim(p1_res)
    p2_res = lim(p2_res)
    
    p1_exp = lim(21 + (p1_score - p2_score) / 10 * 4, lolim=6)  # P1的预期得分
    p2_exp = lim(21 - (p1_score - p2_score) / 10 * 4, lolim=6)  # P2的预期得分
    
    # 实际得分和预期的差
    dif = (p1_res - p1_exp) - (p2_res - p2_exp)
    if _print:
        print("预期比分: {:.1f} : {:.1f}".format(p1_exp, p2_exp))
        print("实际比分: {:.1f} : {:.1f}".format(p1_res, p2_res))
    
    p1_score_new = lim(p1_score + K(p1_score) * dif, uplim=200)
    p2_score_new = lim(p2_score - K(p2_score) * dif, uplim=200)
    if _print:
        print("赛后双方的分数变化：")
        print("P1: {:.1f} --> {:.1f}\nP2: {:.1f} --> {:.1f}".format(p1_score, p1_score_new,
                                                                p2_score, p2_score_new))
    return p1_score_new, p2_score_new


def get_score_2(t1_p1_score, t1_p2_score, t2_p1_score, t2_p2_score, t1_res, t2_res):
    '''
    # 用于计算双打积分
    # Parameters
        t1(2)_p1(2)_score: 比赛前Team1(2)中P1(2)的分数
        t1(2)_res  : 对局中Team1(2)的得分
    # Returns:
        t1(2)_p1(2)_score_new: 比赛后Team1(2)中P1(2)的分数
    '''
    t1 = (t1_p1_score + t1_p2_score) / 2
    t2 = (t2_p1_score + t2_p2_score) / 2
    t1_new, t2_new = get_score_1(t1, t2, t1_res, t2_res)
    
    t1_p1_score_new = lim( t1_p1_score + (t1_new - t1) / 2, uplim=200)
    t1_p2_score_new = lim( t1_p2_score + (t1_new - t1) / 2, uplim=200)
    t2_p1_score_new = lim( t2_p1_score + (t2_new - t2) / 2, uplim=200)
    t2_p2_score_new = lim( t2_p2_score + (t2_new - t2) / 2, uplim=200)
    
    return t1_p1_score_new, t1_p2_score_new, t2_p1_score_new, t2_p2_score_new


def update_data(t1_p1, t1_p2, res1, res2, t2_p1=None, t2_p2=None):
    '''
    # 用于更新数据库中的积分
    # Parameters
        t1(2)_p1(2): Team1(2)中P1(2)的姓名拼音（单打只有Team1）
        res1(2)    : 对局比分
    '''
    if t2_p1 is None:
        data[t1_p1], data[t1_p2] = get_score_1(data[t1_p1], data[t1_p2], res1, res2)
    else:
        data[t1_p1], data[t1_p2], data[t2_p1], data[t2_p2] = \
        get_score_2(data[t1_p1], data[t1_p2], data[t2_p1], data[t2_p2], res1, res2)
    np.save("kiaa_bmt_data.npy", data)
    
def add_player(name, score):
    data[name] = score
    np.save("kiaa_bmt_data.npy", data)
    
def del_player(name):
    del data[name]
    np.save("kiaa_bmt_data.npy", data)

In [2]:
# 读取以字典形式存储的数据 (May)
data = np.load("kiaa_bmt_data00.npy", allow_pickle=True).item()

In [3]:
# 比赛
update_data("WangShun", "FuShuqi", 21,  8, "LiHulin", "LinXiaojing")
update_data("WangShun", "FuShuqi", 21, 13, "LiHulin", "LinXiaojing")
update_data("XuFengwei", "YuNiankun", 21, 14, "LiuChengqi", "ZouSiwei")
update_data("XuFengwei", "YuNiankun", 21, 19, "LiuChengqi", "ZouSiwei")
update_data("LyuCheqiu", "LiHulin", 6, 21)
update_data("LyuCheqiu", "LiHulin", 8, 21)
update_data("YuNiankun", "ZuoPei", 18, 21, "WangHuimei", "LinXiaojing")
update_data("YuNiankun", "ZuoPei", 14, 21, "WangHuimei", "LinXiaojing")
update_data("XuFengwei", "ZhangLulu", 18, 21, "HuangQifeng", "SunShengxiu")
update_data("XuFengwei", "ZhangLulu", 21, 12, "HuangQifeng", "SunShengxiu")
update_data("XuFengwei", "ZhangLulu", 21, 17, "HuangQifeng", "SunShengxiu")
update_data("XueZihan", "Alex", 14, 21, "ShiYangfan", "XuFuxiang")
update_data("XueZihan", "Alex", 10, 21, "ShiYangfan", "XuFuxiang")
update_data("ZhuRui", "ChenSiyuan", 24, 22, "XuFuxiang", "YinGaocheng")
update_data("ZhuRui", "ChenSiyuan", 8, 21, "XuFuxiang", "YinGaocheng")
update_data("ZhuRui", "ChenSiyuan", 8, 21, "XuFuxiang", "YinGaocheng")

# 5/9
update_data("Alex", "ZhangLulu",9,21)
update_data("PanZhiwei", "YinGaocheng", 21, 15, "XuFuxiang", "SunShengxiu")
update_data("XuFengwei", "FuShuqi", 21, 13, "WangShun", "WangHuimei")

# 5/11
update_data("XuFengwei", "PanZhiwei", 16, 21)

# 5/16
update_data("PanZhiwei", "LiuChengqi",21,13)
update_data("PanZhiwei", "LiuChengqi",21,8)
update_data("XuFuxiang", "ShiYangfan",10,21)
update_data("XuFuxiang", "ShiYangfan",11,21)
add_player("FengQikang",84.37)
del_player("PangYuxuan")

# 5/17
update_data("PanZhiwei", "XuFuxiang",21,18,"WangShun", "LyuCheqiu")
update_data("PanZhiwei", "XuFuxiang",13,21,"WangShun", "LyuCheqiu")
update_data("PanZhiwei", "XuFuxiang",21,19,"WangShun", "LyuCheqiu")
update_data("PanZhiwei", "XuFuxiang",15,21,"WangShun", "ZouSiwei")
update_data("PanZhiwei", "XuFuxiang",21,19,"WangShun", "ZouSiwei")
update_data("PanZhiwei", "XuFuxiang",16,21,"WangShun", "ZouSiwei")
update_data("SunShengxiu", "FengQikang", 23, 21)

# 5/25
update_data("XuFengwei","Alex",21,9)
update_data("WangShun","LyuCheqiu",21,19,"PanZhiwei","FuShuqi")
update_data("XuFengwei","Alex",11,21,"PanZhiwei","FuShuqi")
update_data("XuFengwei","Alex",10,21,"WangShun","LyuCheqiu")
update_data("WangShun","ZouSiwei",21,19,"PanZhiwei","FuShuqi")
update_data("WangShun","XuFengwei",15,21,"PanZhiwei","FuShuqi")
update_data("LyuCheqiu","Alex",21,8)
update_data("XuFengwei","Alex",15,21,"WangShun","LyuCheqiu")
update_data("WangShun","ZouSiwei",16,21,"PanZhiwei","FuShuqi")
update_data("XuFengwei","Alex",12,21,"PanZhiwei","LyuCheqiu")
update_data("XuFengwei","ZouSiwei",8,21,"PanZhiwei","FuShuqi")
update_data("XuFengwei","Alex",12,21,"PanZhiwei","LyuCheqiu")
update_data("WangShun","ZouSiwei",18,21,"PanZhiwei","FuShuqi")

sorted(data.items(), key=lambda x:x[1], reverse=1)

[('WangShun', 143.97580015621077),
 ('PanZhiwei', 140.70815318453856),
 ('XuFengwei', 124.39642976784998),
 ('LiHulin', 118.56331092500001),
 ('LiuChengqi', 115.4045295904021),
 ('FeiQinyue', 100),
 ('ShiYangfan', 85.210017296641),
 ('FuShuqi', 84.23520490648608),
 ('WangHuimei', 78.54615132370895),
 ('FengQikang', 76.8483887568896),
 ('SunShengxiu', 76.68721304555858),
 ('LyuCheqiu', 71.20298173745712),
 ('HuangQifeng', 66.86638237824465),
 ('XueZihan', 66.279815),
 ('YuNiankun', 64.5812487929062),
 ('ZhangLulu', 58.1005929758376),
 ('XuFuxiang', 55.419765831143565),
 ('ZouSiwei', 53.53083233038423),
 ('LinXiaojing', 46.21477418594659),
 ('Alex', 40.54072911111483),
 ('ZuoPei', 34.762363792906186),
 ('YinGaocheng', 25.271768112598),
 ('ZhuRui', 23.543825117327227),
 ('YueNannan', 10),
 ('ChenSiyuan', 3.543825117327227)]

## .csv to .npy

In [62]:
import pandas as pd
import numpy as np

data = pd.read_csv("data.csv")
dict1 = dict(zip(data['name'],data['score']))
np.save('kiaa_bmt_data.npy', dict1)

## .npy to .csv

In [4]:
import pandas as pd
d = pd.DataFrame(columns=["Name", "Score"])
d["Name"] = data.keys()
d["Score"] = np.round(np.fromiter(data.values(), dtype=float), 1)
d= d.sort_values("Score", ascending=0)
d.to_csv("data.csv", index=None)

## github

In [5]:
for i in sorted(data.items(), key=lambda x:x[1], reverse=1):
    print(i[0], round(i[1],1), "\n")

WangShun 144.0 

PanZhiwei 140.7 

XuFengwei 124.4 

LiHulin 118.6 

LiuChengqi 115.4 

FeiQinyue 100 

ShiYangfan 85.2 

FuShuqi 84.2 

WangHuimei 78.5 

FengQikang 76.8 

SunShengxiu 76.7 

LyuCheqiu 71.2 

HuangQifeng 66.9 

XueZihan 66.3 

YuNiankun 64.6 

ZhangLulu 58.1 

XuFuxiang 55.4 

ZouSiwei 53.5 

LinXiaojing 46.2 

Alex 40.5 

ZuoPei 34.8 

YinGaocheng 25.3 

ZhuRui 23.5 

YueNannan 10 

ChenSiyuan 3.5 

